# Problem 1

In [9]:
import math

In [6]:
# convert data from txt file to dict
dicts = {}
i = 0
with open("W2V_150.txt") as f:
    for lines in f:
        i += 1
        if i >= 3:
            line = lines.split()
            key = line[0]
            line.pop(0)
            val = line
            dicts[str(key)] = val
            for i in range(0, len(val)):
                val[i] = float(val[i])

In [5]:
def cosine_similarity(v1,v2):
    xx, xy, yy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        xx += x*x
        yy += y*y
        xy += x*y
    return xy/math.sqrt(xx*yy)

In [25]:
from prettytable import PrettyTable
table = PrettyTable(['Word 1', 'Word 2', 'Cosine_similarity'])

In [26]:
from scipy import stats
i = 0
with open("ViSim-400.txt") as f:
    for lines in f:
        i += 1
        if i > 1:
            line = lines.split()
            if line[0] in dicts.keys() and line[1] in dicts.keys():
                v1 = dicts.get(line[0])        
                v2 = dicts.get(line[1])
                table.add_row([line[0], line[1], cosine_similarity(v1, v2)])
            else:
                table.add_row([line[0], line[1], 'NaN'])
print(table)

+---------------------+---------------------+------------------------+
|        Word 1       |        Word 2       |   Cosine_similarity    |
+---------------------+---------------------+------------------------+
|         biến        |         ngập        | -0.0049123394696700545 |
|     nhà_thi_đấu     |         nhà         |  0.08252318329211768   |
|         động        |         tĩnh        |   0.2770859598682775   |
|        khuyết       |          ưu         |  0.17679862835626708   |
|       cõi_tục       |        cõi_âm       |          NaN           |
|       thủ_pháp      |      biện_pháp      |  0.40236612919430603   |
|      kết_duyên      |      thành_hôn      |   0.4630084020140725   |
|       cấp_tiến      |       bảo_thủ       |   0.2569470088996125   |
|       nước_lớn      |      nguy_hiểm      |   0.1851920224021162   |
|         hoa         |         nhị         |  0.22306960452548538   |
|       bất_lợi       |      thuận_lợi      |   0.5348913029963387   |
|     

# Problem 2

In [35]:
import operator

k = int(input())
word = input()
sim_mp = {}
if word in dicts.keys():
    for key in dicts:
        if (word != key):
            sim_mp[key] = cosine_similarity(dicts.get(word), dicts.get(key))
    sim_mp = sorted(sim_mp.items(), key=operator.itemgetter(1), reverse=True)
    key_list = list(sim_mp)
    for i in range(k):
        print(key_list[i])

10
tiết_lộ
('hé_lộ', 0.8141896754726131)
('bật_mí', 0.6960303061638391)
('công_bố', 0.5812819713581816)
('xác_nhận', 0.5785359220947899)
('thổ_lộ', 0.5612686986844789)
('phát_hiện_ra', 0.5585788908739554)
('loan_báo', 0.5559826694440024)
('khám_phá_ra', 0.5496182853839372)
('thú_nhận', 0.5479432813714646)
('úp_mở', 0.5438636450711214)


# Problem 3

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

In [48]:
w2v_file = open("W2V_150.txt", encoding='utf8')
antonym_file = open("Antonym_vietnamese.txt", encoding='utf8')
synonym_file = open("Synonym_vietnamese.txt", encoding='utf8')

In [49]:
antonym_lines = [line for line in antonym_file.readlines()]

In [50]:
synonym_lines = [line for line in synonym_file.readlines()]

In [51]:
words = [line for line in w2v_file.readlines()]
num_of_words = words[0]
dims = words[1]
words = words[2:]

In [52]:
arr_words = []
embedding = {}

In [53]:
for idx, w in enumerate(words):
    arr_words = words[idx].split(' ')
    word = arr_words[0]
    weight = []
    for i in arr_words[2:len(arr_words)-1]:
        weight.append(float(i))
    embedding[word] = np.array(weight)

In [54]:
antonym_word1 = []
antonym_word2 = []
synonym_word1 = []
synonym_word2 = []

for idx, w in enumerate(antonym_lines):
    arr_words = antonym_lines[idx].split(' ')
    antonym_word1.append(arr_words[0])
    antonym_word2.append(arr_words[1].split('\n')[0])

for idx, w in enumerate(synonym_lines):
    arr_words = synonym_lines[idx].split(' ')
    synonym_word1.append(arr_words[0])
    synonym_word2.append(arr_words[1].split('\n')[0])

In [70]:
word_not_in_w2v = []
for w in antonym_word1:
    if w not in dicts.keys():
        word_not_in_w2v.append(w)
for w in antonym_word2:
    if w not in dicts.keys():
        word_not_in_w2v.append(w)
for w in synonym_word1:
    if w not in dicts.keys():
        word_not_in_w2v.append(w)
for w in synonym_word2:
    if w not in dicts.keys():
        word_not_in_w2v.append(w)

In [71]:
word_not_in_w2v = list(dict.fromkeys(word_not_in_w2v))
len(word_not_in_w2v)

4633

In [88]:
missing_words_dict = {}
for w in word_not_in_w2v:
    missing_words_dict[w] = np.random.normal(0, 1, 150).tolist()

In [90]:
dicts.update(missing_words_dict)

In [93]:
len(dicts.keys())

81654

In [123]:
X_train = []
y_train = []
for i in range(0, len(synonym_word1)):
    X_train.append(cosine_similarity(dicts[synonym_word1[i]], dicts[synonym_word2[i]]))
    y_train.append(1)
for i in range(0, len(antonym_word1)):
    X_train.append(cosine_similarity(dicts[antonym_word1[i]], dicts[antonym_word1[i]]))
    y_train.append(0)


In [124]:
X_train = np.asarray(X_train)

In [139]:
X_train = X_train.reshape(-1, 1)

In [141]:
X_train.shape

(13562, 1)

In [142]:
y_train = np.asarray(y_train)

In [143]:
y_train.shape


(13562,)

In [144]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [147]:
n_test = pd.read_csv("400_noun_pairs.txt", sep='\t')
v_test = pd.read_csv('400_verb_pairs.txt',sep='\t')
a_test = pd.read_csv('600_adj_pairs.txt', sep='\t')
df_test = pd.concat([n_test, v_test, a_test], ignore_index=True)

In [150]:
ant_test = df_test.loc[df_test['Relation']=='ANT'].reset_index() #700x4
syn_test = df_test.loc[df_test['Relation']=='SYN'].reset_index() #700x4
syn_test

,index,Word1,Word2,Relation
0,3,dây,dọc,SYN
1,4,bằng_cớ,chứng_cớ,SYN
2,7,gông_cùm,gông_xiềng,SYN
3,8,nhân_tài,tài_năng,SYN
4,11,dạ_hương,dạ_lý_hương,SYN
...,...,...,...,...
695,1384,vất_vả,nặng_nhọc,SYN
696,1389,dong_dỏng,thon_thả,SYN
697,1390,tươm,tướp,SYN
698,1391,trống_không,trống_trơn,SYN


In [162]:
X_test = []
y_test = []
for i in range(0, 700):
    X_test.append(cosine_similarity(dicts[syn_test['Word1'][i]], dicts[syn_test['Word2'][i]]))
    y_test.append(1)
    X_test.append(cosine_similarity(dicts[ant_test['Word1'][i]], dicts[ant_test['Word2'][i]]))
    y_test.append(0)

In [163]:
X_test = np.asarray(X_test).reshape(-1, 1)

In [164]:
X_test.shape

(1400, 1)

In [165]:
y_test = np.asarray(y_test)
y_test.shape

(1400,)

In [166]:
y_pred = model.predict(X_test)

In [168]:
score = model.score(X_test, y_test)
print(score)

0.4992857142857143
